# 1. Historical Model


### Required inputs:
- Soils
- Stage-Storage
- Historical Climate
- Calibration Data


### Results: 
- Model Data
- Timeseries
- Model Parameters

To run, specify filepath of repository folder on your local machine. 

The filepath './' denotes the root folder relative to this notebook, so unless you are referencing input files in a different location than this notebook file, you can leave this as is. 

The input files must maintain the same general subfolder structure. 


In [1]:

################### user inputs ###############################
# if you are running input folder in the same directory as this notebook file, use './', otherwise change to the desired file path
repo_folder = './'  


## specify the input filename of the main_inputs file; do not include file extension
master_input_filename = 'main_inputs'





###############################################################
import os
os.chdir(repo_folder)
import importlib

from pondit.load_data import *
from pondit.calc_ET import *
from pondit.calc_model import *
from pondit.plot_model import *

import pandas
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

%matplotlib inline

folder_in = repo_folder + '/inputs/'
folder_out = repo_folder + '/results/'

### load input data common to all sites
scalars = pandas.read_csv(folder_in + master_input_filename + '.csv', index_col=0)

site_list = scalars.index
site_list = scalars.loc[scalars['status'] != 'done', :].index


for site in site_list:
    sws_calc_dict = {}
    print(site)
    ## load data
    stage_storage, soils, data_hist, last_hist_date, calib_data = load_data(scalars, site, folder_in)

    
    bc_calc = calc_ET(data_hist, scalars, site, repo_folder)

    sws_calc, out, scalars_out = calc_pondit(bc_calc, scalars, site, stage_storage, soils, repo_folder, calib_data)
    sws_calc.to_csv(folder_out + '/data/'+ site + '_data_hist.csv')
    scalars_out.to_csv(folder_out + 'data/' + site + '_model_params_hist.csv')
    plot_model(sws_calc, scalars, site, folder_out)
        



Pond_TA
            calib_wse_ft_area
date                         
2007-06-01         807.476968
2010-09-01         804.844308
2011-06-01         807.387368
2011-09-01         806.679400
2012-02-01         807.072421
2014-02-01         803.012658
2016-04-01         807.477305
2018-05-01         807.420547
Blaney Criddle fit parameters: a =  1.1312140857983362  b =  -1.6547222539623263
[0.05 0.05 2.   1.5  0.1  0.01 0.5  0.7  0.2  0.5  0.4 ] sum squared error:  14.673378760853193


KeyboardInterrupt: 